In [1]:
#import zipfile
#zip = zipfile.ZipFile('/home/blueee/DL_Homework_1.zip')
#zip.extractall(path='/home/blueee/DL_Homework_1/')
#zip.close()

In [2]:
import os
import cv2
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score

def resize_with_aspect_ratio(image, target_size):
    height, width = image.shape[:2]
    target_width, target_height = target_size
    width_ratio = target_width / width
    height_ratio = target_height / height
    ratio = min(width_ratio, height_ratio)
    resized_image = cv2.resize(image, (int(width * ratio), int(height * ratio)), interpolation=cv2.INTER_LINEAR)
    return resized_image
    
def extract_color_histogram(image, bins=(8, 8, 8)):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

def load_img(file_path, target_size):
    f = open(file_path)
    lines = f.readlines()
    imgs, lab = [], []
    for line in lines:
        fn, label = line.strip().split(' ')
        image = cv2.imread(fn)
        resized_image = resize_with_aspect_ratio(image, target_size)
        hist = extract_color_histogram(resized_image)
        imgs.append(hist)
        lab.append(int(label))
    imgs = np.asarray(imgs, np.float32)
    lab = np.asarray(lab, np.int32)
    return imgs, lab 

In [3]:
dd=os.listdir('/home/blueee/DL_Homework_1/TIN')
f1 = open('train.txt', 'w')
f2 = open('test.txt', 'w')
for i in range(len(dd)):
    d2 = os.listdir ('./TIN/%s/images/'%(dd[i]))
    for j in range(len(d2)-2):
        str1='TIN/%s/images/%s'%(dd[i], d2[j])
        f1.write("%s %d\n" % (str1, i))
    str1='TIN/%s/images/%s'%(dd[i], d2[-1])
    f2.write("%s %d\n" % (str1, i))

f1.close()
f2.close()

In [4]:
target_size = (256, 256)
x, y = load_img('train.txt', target_size)
tx, ty = load_img('test.txt', target_size)

In [5]:
print("訓練數據的圖像數據形狀:", x.shape)
print("訓練數據的標籤數據形狀:", y.shape)
print("測試數據的圖像數據形狀:", tx.shape)
print("測試數據的標籤數據形狀:", ty.shape)

訓練數據的圖像數據形狀: (99600, 512)
訓練數據的標籤數據形狀: (99600,)
測試數據的圖像數據形狀: (200, 512)
測試數據的標籤數據形狀: (200,)


In [6]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier(n_neighbors=5, algorithm='auto')
knn_classifier.fit(x, y)

y_pred = knn_classifier.predict(tx)

accuracy = accuracy_score(ty, y_pred)
f1 = f1_score(ty, y_pred, average='weighted')

print(f"color_histogram + KNN 准确率: {accuracy:.4f}")
print(f"F1分数: {f1:.4f}")

color_histogram + KNN 准确率: 0.0450
F1分数: 0.0364
CPU times: user 8.88 s, sys: 2.85 ms, total: 8.88 s
Wall time: 550 ms


In [7]:
%%time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)

rf_model.fit(x, y)

y_pred = rf_model.predict(tx)

accuracy = accuracy_score(ty, y_pred)
f1 = f1_score(ty, y_pred, average='weighted')

print(f"color_histogram + RF准确率: {accuracy:.4f}")
print(f"F1分数: {f1:.4f}")

color_histogram + RF准确率: 0.1150
F1分数: 0.0752
CPU times: user 17min 40s, sys: 18 s, total: 17min 58s
Wall time: 29.8 s


In [8]:
%%time
from sklearn.model_selection import train_test_split

svm_classifier = LinearSVC(C=0.8, dual=False, verbose=0)
svm_classifier.fit(x, y)

y_pred = svm_classifier.predict(tx)

accuracy = accuracy_score(ty, y_pred)
f1 = f1_score(ty, y_pred, average='weighted')

print(f"color_histogram + svm準確率: {accuracy:.4f}")
print(f"F1分數: {f1:.4f}")

color_histogram + svm準確率: 0.0600
F1分數: 0.0281
CPU times: user 41min 24s, sys: 22.2 s, total: 41min 46s
Wall time: 41min 45s
